Preprocess

In [1]:
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import segmentation_models_pytorch as smp
from torchvision import transforms
from tqdm import tqdm
from PIL import Image

c:\Users\RUJUTA\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Dataloader for Change Detection
class ChangeDetectionDataset(Dataset):
    def __init__(self, t2019_dir, t2024_dir, mask_dir, classes, transform=None):
        self.t2019_dir = t2019_dir
        self.t2024_dir = t2024_dir
        self.mask_dir = mask_dir
        self.classes = classes
        self.transform = transform
        self.t2019_paths = sorted([f for f in os.listdir(t2019_dir) if f.endswith('.tif')])
        self.t2024_paths = sorted([f for f in os.listdir(t2024_dir) if f.endswith('.tif')])
        self.mask_paths = sorted([f for f in os.listdir(mask_dir) if f.endswith('.tif')])

    def __len__(self):
        return len(self.t2019_paths)

    def __getitem__(self, index):
        img_t2019 = np.array(Image.open(os.path.join(self.t2019_dir, self.t2019_paths[index])), dtype=np.float32) / 255.0
        img_t2024 = np.array(Image.open(os.path.join(self.t2024_dir, self.t2024_paths[index])), dtype=np.float32) / 255.0
        mask = np.array(Image.open(os.path.join(self.mask_dir, self.mask_paths[index])), dtype=np.int64)

        # Stack time series images for model input
        image = np.concatenate([img_t2019, img_t2024], axis=-1)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image, mask = augmented['image'], augmented['mask']

        image = torch.from_numpy(image).permute(2, 0, 1)
        mask = torch.from_numpy(mask)

        return image, mask


In [5]:
# Define class labels
CLASSES = ['no_change', 'increase_vegetation', 'decrease_vegetation']

# DataLoader Setup
train_dataset = ChangeDetectionDataset(
    t2019_dir="CDTest/Train/Images/T2019",
    t2024_dir="CDTest/Train/Images/T2024",
    mask_dir="CDTest/Train/cd2_Output",
    classes=CLASSES
)

val_dataset = ChangeDetectionDataset(
    t2019_dir="CDTest/Val/Images/T2019",
    t2024_dir="CDTest/Val/Images/T2024",
    mask_dir="CDTest/Val/cd2_Output",
    classes=CLASSES
)

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=32,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32,shuffle=False)


In [6]:
def describe_loader(loader_type):
    print("batch size: ",loader_type.batch_size)
    print("shape: ", loader_type.dataset[0][0].shape, loader_type.dataset[0][1].shape)
    print(np.unique(loader_type.dataset[0][1]))
    print("number of images: ", len(loader_type.dataset))
    print("classes: ", loader_type.dataset.classes)

print("------------train------------")
describe_loader(train_loader)
print("------------val------------")
describe_loader(val_loader)

------------train------------
batch size:  32
shape:  torch.Size([6, 512, 512]) torch.Size([512, 512])
[0 1 2]
number of images:  9
classes:  ['no_change', 'increase_vegetation', 'decrease_vegetation']
------------val------------
batch size:  32
shape:  torch.Size([6, 512, 512]) torch.Size([512, 512])
[0 1 2]
number of images:  2
classes:  ['no_change', 'increase_vegetation', 'decrease_vegetation']


Model

In [10]:
# Model Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = smp.DeepLabV3Plus(
    encoder_name="resnet18",
    encoder_weights="imagenet",
    classes=3,
    activation=None,
    in_channels=6  # Stacked images for T2019 and T2024
)
model.to(device)

# Hyperparameters
epochs = 2
lr = 1e-4
batch_size = 8
num_classes = 3

# Loss and Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [11]:
# Training Loop
for epoch in range(epochs):
    model.train()
    train_loss = 0

    for images, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Training]"):
        images, masks = images.to(device), masks.to(device)

        outputs = model(images)
        loss = loss_fn(outputs, masks.long())
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}")

    # Validation Loop
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for images, masks in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Validation]"):
            images, masks = images.to(device), masks.to(device)

            outputs = model(images)
            loss = loss_fn(outputs, masks.long())
            val_loss += loss.item()

    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}")


Epoch 1/2 [Training]: 100%|██████████| 1/1 [00:45<00:00, 45.79s/it]


Epoch 1, Train Loss: 0.9032


Epoch 1/2 [Validation]: 100%|██████████| 1/1 [00:03<00:00,  3.72s/it]


Epoch 1, Validation Loss: 1.1420


Epoch 2/2 [Training]: 100%|██████████| 1/1 [00:39<00:00, 39.21s/it]


Epoch 2, Train Loss: 0.8823


Epoch 2/2 [Validation]: 100%|██████████| 1/1 [00:02<00:00,  2.73s/it]

Epoch 2, Validation Loss: 1.1354


In [ ]:
# Save Model
torch.save(model.state_dict(), "change_detection_model.pth")
print("Model saved.")

Test

Display